### Data Processing ###

In [16]:
import os
import random
import shutil

# Directories
images_dir = './../datasets/dataset_c3v1/images'  # Path to the train images folder
labels_dir = './../datasets/dataset_c3v1/labels/train'  # Path to the labels folder
train_images_dir = './../yolo_datasets/dataset_c3v1_processed/train_split/images'  # New train images folder
train_labels_dir = './../yolo_datasets/dataset_c3v1_processed/train_split/labels'  # New train labels folder
test_images_dir = './../yolo_datasets/dataset_c3v1_processed/test_split/images'  # New test images folder
test_labels_dir = './../yolo_datasets/dataset_c3v1_processed/test_split/labels'  # New test labels folder

# Create the new directories for train/test splits
os.makedirs(train_images_dir, exist_ok=True)
os.makedirs(train_labels_dir, exist_ok=True)
os.makedirs(test_images_dir, exist_ok=True)
os.makedirs(test_labels_dir, exist_ok=True)

# Get all image files in the train directory
image_files = [f for f in os.listdir(images_dir) if f.endswith('.jpeg')]

# Shuffle the image files for randomness
random.shuffle(image_files)

# Set the ratio for the split (80% for train and 20% for test)
train_ratio = 0.8
split_index = int(len(image_files) * train_ratio)

# Select train and test files
train_files = image_files[:split_index]
test_files = image_files[split_index:]

# Move files to respective folders
for file in train_files:
    # Move image
    shutil.copy(os.path.join(images_dir, file), os.path.join(train_images_dir, file))
    # Move label
    shutil.copy(os.path.join(labels_dir, file.replace('.jpeg', '.txt')), os.path.join(train_labels_dir, file.replace('.jpeg', '.txt')))

for file in test_files:
    # Move image
    shutil.copy(os.path.join(images_dir, file), os.path.join(test_images_dir, file))
    # Move label
    shutil.copy(os.path.join(labels_dir, file.replace('.jpeg', '.txt')), os.path.join(test_labels_dir, file.replace('.jpeg', '.txt')))

# Write new train.txt and test.txt
with open('./../yolo_datasets/dataset_c3v1_processed/train_split/train.txt', 'w') as f:
    for file in train_files:
        f.write(f'{train_images_dir}/{file}\n')

with open('./../yolo_datasets/dataset_c3v1_processed/test_split/test.txt', 'w') as f:
    for file in test_files:
        f.write(f'{test_images_dir}/{file}\n')

print("Train-test split completed!")


Train-test split completed!


### Model Training ###


In [38]:
from ultralytics import YOLO

# Load the YOLOv8 segmentation model (pretrained)
model = YOLO("yolov8n-seg.pt")

# Train model
model.train(
    data="../yolo_datasets/dataset_c3v1_processed/data.yaml",
    epochs=50,
    imgsz=640,
    batch=8,
    device="cpu"
)

Ultralytics 8.3.75 🚀 Python-3.12.9 torch-2.6.0 CPU (Apple M2 Pro)
engine/trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=../yolo_datasets/dataset_c3v1_processed/data.yaml, epochs=50, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True,

train: Scanning /Users/sonny/Desktop/Code/STIK_GUI/yolo_datasets/dataset_c3v1_processed/train_split/Users/sonny/Desktop/Code/STIK_GUI/yolo_datasets/dataset_c3v1_processed/train_split/yolo_datasets/dataset_c3v1_processed/train_split/labels... 0 images, 0 backgrounds, 86 corrupt: 100%|██████████| 86/86 [00:00<00:00, 36183.18it/s]

train: WARNING ⚠️ /Users/sonny/Desktop/Code/STIK_GUI/yolo_datasets/dataset_c3v1_processed/train_split/./Users/sonny/Desktop/Code/STIK_GUI/yolo_datasets/dataset_c3v1_processed/train_split/yolo_datasets/dataset_c3v1_processed/train_split/images/11042024_CG_49koseries_invasionassay 49 and 188 m4k4 ko_Sequence 001_C_3_Mosaic001_Merged_t00_ch00.jpeg: ignoring corrupt image/label: [Errno 2] No such file or directory: '/Users/sonny/Desktop/Code/STIK_GUI/yolo_datasets/dataset_c3v1_processed/train_split/./Users/sonny/Desktop/Code/STIK_GUI/yolo_datasets/dataset_c3v1_processed/train_split/yolo_datasets/dataset_c3v1_processed/train_split/images/11042024_CG_49koseries_invasionassay 49 and 188 m4k4 ko_Sequence 001_C_3_Mosaic001_Merged_t00_ch00.jpeg'
train: WARNING ⚠️ /Users/sonny/Desktop/Code/STIK_GUI/yolo_datasets/dataset_c3v1_processed/train_split/./Users/sonny/Desktop/Code/STIK_GUI/yolo_datasets/dataset_c3v1_processed/train_split/yolo_datasets/dataset_c3v1_processed/train_split/images/11042024_CG

ValueError: not enough values to unpack (expected 3, got 0)

In [27]:
metrics = model.val()
print(metrics)

Ultralytics 8.3.75 🚀 Python-3.12.9 torch-2.6.0 CPU (Apple M2 Pro)
YOLOv8n-seg summary (fused): 195 layers, 3,404,320 parameters, 0 gradients, 12.6 GFLOPs

Dataset 'coco.yaml' images not found ⚠️, missing path '/Users/sonny/Desktop/Code/STIK_GUI/stik_ai/datasets/coco/val2017.txt'


100%|██████████| 169M/169M [00:06<00:00, 28.4MB/s] 
Unzipping /Users/sonny/Desktop/Code/STIK_GUI/stik_ai/datasets/coco2017labels-segments.zip to /Users/sonny/Desktop/Code/STIK_GUI/stik_ai/datasets/coco...:  76%|███████▌  | 93191/122232 [00:09<00:02, 9915.83file/s] 


KeyboardInterrupt: 

In [ ]:
results = model("path/to/test_image.jpg")
results.show()


In [ ]:
import os

input_folder = "path/to/new_images"
output_folder = "path/to/annotated_images"
os.makedirs(output_folder, exist_ok=True)

# Run inference on all images in the folder
results = model([os.path.join(input_folder, img) for img in os.listdir(input_folder)])

# Save annotated images
for i, r in enumerate(results):
    r.save(os.path.join(output_folder, f"annotated_{i}.jpg"))  # Saves output images

areas = []

# Iterate through results and calculate area for segmentation masks
for r in results:
    if r.masks:  # Check if masks exist
        for mask in r.masks:
            area = mask.area()  # Get the area of the segmentation mask
            areas.append(area)

print(areas)  # Print the array of areas
